**Trabalho Prático Machine Learning**

**LUIZA CASTELAR RODRIGUES PEREIRA**


Importação e leitura dos dados

In [ ]:
# bibliotecas para análise
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import plotly.express as px

#bibliotecas para Naive Bayes e Random Forest
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

In [ ]:
download_url = 'https://drive.google.com/uc?export=download&id='
url_drive_file = 'https://docs.google.com/spreadsheets/d/1eCsMmmW2m7RLLfR-9uBOPEiH3N6d3nG4/edit?usp=sharing&ouid=100851951330604893448&rtpof=true&sd=true'
download_path = download_url + url_drive_file.split('/')[-2]

eleicao = pd.read_excel(download_path, encoding='utf-8')

eleicao.head()

<b>Descrição dos dados</b>

A tabela possui 81.785 registros ao todo e 63 colunas. 

A análise real de nulos será feita após a remoção das colunas que não usarei, mas já da para ver que as colunas "NM_COLIGACAO", "DT_NASCIMENTO" e "NR_IDADE_DATA_POSSE" possuem registros nulos.


In [ ]:
eleicao.info()

Verificando algumas colunas individualmente que suspeito terem valores únicos, para ver se os valores encontrados em colunas que em primeiro momento não vejo utilidade, tenham de fato um valor interessante para ser analisado ou descartado:

In [ ]:
eleicao['DT_GERACAO'].value_counts()

In [ ]:
eleicao['HH_GERACAO'].value_counts()

In [ ]:
eleicao['ANO_ELEICAO'].value_counts()

In [ ]:
eleicao['CD_TIPO_ELEICAO'].value_counts()

In [ ]:
eleicao['NM_TIPO_ELEICAO'].value_counts()

In [ ]:
eleicao['NR_TURNO'].value_counts()

In [ ]:
eleicao['CD_ELEICAO'].value_counts()

In [ ]:
eleicao['DS_ELEICAO'].value_counts()

In [ ]:
Counter(eleicao['DT_ELEICAO'])

In [ ]:
Counter(eleicao['TP_ABRANGENCIA'])

In [ ]:
Counter(eleicao['SG_UF'])

In [ ]:
Counter(eleicao['CD_CARGO'])

In [ ]:
Counter(eleicao['DS_CARGO'])

In [ ]:
Counter(eleicao['CD_SITUACAO_CANDIDATURA'])

In [ ]:
Counter(eleicao['DS_SITUACAO_CANDIDATURA'])

In [ ]:
Counter(eleicao['CD_DETALHE_SITUACAO_CAND'])

In [ ]:
eleicao['DS_DETALHE_SITUACAO_CAND'].value_counts()

In [ ]:
Counter(eleicao['CD_GRAU_INSTRUCAO'])

In [ ]:
eleicao['DS_GRAU_INSTRUCAO'].value_counts()

In [ ]:
Counter(eleicao['CD_ESTADO_CIVIL'])

In [ ]:
eleicao['DS_ESTADO_CIVIL'].value_counts()

In [ ]:
Counter(eleicao['CD_GENERO'])

In [ ]:
eleicao['DS_GENERO'].value_counts()

In [ ]:
Counter(eleicao['CD_MUNICIPIO_NASCIMENTO'])

In [ ]:
Counter(eleicao['CD_NACIONALIDADE'])

In [ ]:
eleicao['DS_NACIONALIDADE'].value_counts()

In [ ]:
eleicao['ST_CANDIDATO_INSERIDO_URNA'].value_counts()

In [ ]:
Counter(eleicao['CD_COR_RACA'])

In [ ]:
eleicao['DS_COR_RACA'].value_counts()

In [ ]:
eleicao['ST_REELEICAO'].value_counts()

In [ ]:
Counter(eleicao['NR_PROTOCOLO_CANDIDATURA'])

In [ ]:
Counter(eleicao['CD_SIT_TOT_TURNO'])

In [ ]:
eleicao['DS_SIT_TOT_TURNO'].value_counts()

Após análise anterior dos valores de algumas colunas, seleciono as colunas que julgo necessárias para a solução do problema:

In [ ]:
eleicao1 = pd.DataFrame()

#colunas auxiliares para remover/consertar algum ou vários registros/candidatos
eleicao1['ST_CANDIDATO_INSERIDO_URNA'] = eleicao['ST_CANDIDATO_INSERIDO_URNA']

eleicao1['DS_DETALHE_SITUACAO_CAND'] = eleicao['DS_DETALHE_SITUACAO_CAND'] 

eleicao1['DS_SITUACAO_CANDIDATURA'] = eleicao['DS_SITUACAO_CANDIDATURA'] 

eleicao1['SQ_CANDIDATO'] = eleicao['SQ_CANDIDATO'] 

eleicao1['NR_TURNO'] = eleicao['NR_TURNO'] 

#colunas para análise extra (entender a base)
eleicao1['NM_CANDIDATO'] = eleicao['NM_CANDIDATO']

eleicao1['TP_AGREMIACAO'] = eleicao['TP_AGREMIACAO'] 

eleicao1['DS_NACIONALIDADE'] = eleicao['DS_NACIONALIDADE'] 

eleicao1['NR_IDADE_DATA_POSSE'] = eleicao['NR_IDADE_DATA_POSSE'] 

eleicao1['NM_SOCIAL_CANDIDATO'] = eleicao['NM_SOCIAL_CANDIDATO'] 

#colunas que quero analisar
eleicao1['DS_GENERO'] = eleicao['DS_GENERO']  

eleicao1['DS_ESTADO_CIVIL'] = eleicao['DS_ESTADO_CIVIL'] 

eleicao1['DS_GRAU_INSTRUCAO'] = eleicao['DS_GRAU_INSTRUCAO'] 

eleicao1['DS_COR_RACA'] = eleicao['DS_COR_RACA'] 

eleicao1['DS_CARGO'] = eleicao['DS_CARGO'] 

# label
eleicao1['DS_SIT_TOT_TURNO'] = eleicao['DS_SIT_TOT_TURNO']

Saio de um dataframe que tinha 63 colunas para um que tem 16.

In [ ]:
eleicao1.shape

Dataframe final:

In [ ]:
eleicao1.head()

Análisando os motivos que levaram os candidatos a não serem inseridos na urna:

In [ ]:
qtdNaoInseridos = np.where(eleicao1['ST_CANDIDATO_INSERIDO_URNA'] != 'SIM')

porqueNaoInseridos = eleicao1.loc[qtdNaoInseridos]['DS_DETALHE_SITUACAO_CAND'].unique()

print "Quantidade de candidatos não inseridos na urna: ", len(qtdNaoInseridos[0])

print"Motivos da nao inserção:", porqueNaoInseridos

print "\nAnálise gráfica de porcentagem sobre os NÃO aceitos:"

px.pie(eleicao.loc[qtdNaoInseridos], names='DS_DETALHE_SITUACAO_CAND')

Exclusão dos candidatos que não foram inseridos na urna:

In [ ]:
eleicao2 = eleicao1.loc[np.where(eleicao1['ST_CANDIDATO_INSERIDO_URNA'] == 'SIM')]

Verificando quantos candidatos ainda estão com o valor "Inapto" e por quais motivos desse valor:

In [ ]:
print "Quantos inaptos ainda existem na base:", len(np.where(eleicao2['DS_SITUACAO_CANDIDATURA'] == 'INAPTO')[0]),'\n'

inaptos = eleicao2.iloc[np.where(eleicao2['DS_SITUACAO_CANDIDATURA'] == 'INAPTO')]

inaptos.head()

Na base tem dados de candidatos duplicados devido à 2º turno. 

Removemos os registros relativos a esses candidatos, deixando os registros de 2º turno aonde tem a decisão final de eleição.

In [ ]:
eleicao3.select_dtypes(include=['object']).describe()

In [ ]:
condicao = np.where(eleicao2.duplicated(subset=['SQ_CANDIDATO'], keep=False) == True)

duplicados = (eleicao2.iloc[condicao]).sort_values(by=['NM_CANDIDATO'])

print "Quantos duplicados tem a base em função de 2º turno: ", len(duplicados)

duplicadosExcluir = duplicados.iloc[np.where(duplicados['NR_TURNO'] == 1)]

indices = list(set(eleicao2.index) - set(duplicadosExcluir.index))

eleicao3 = eleicao2.loc[indices]

**Tratando o label**
*   Como temos somente 2 registros com nulo, descartaremos eles.
*   Agruparemos todos os valores com "eleito" na composição do nome para somente "eleito"



In [ ]:
eleicao3['DS_SIT_TOT_TURNO'].value_counts()

In [ ]:
titulo = eleicao3['DS_SIT_TOT_TURNO']

eleito = []

for el in titulo:
  if (el != 'NÃO ELEITO'.decode('utf-8')) and ('ELEITO' in el):
    eleito.append('ELEITO')
  elif el == '#NULO#':
    eleito.append(None)
  else:
    eleito.append(el)

eleicao3['DS_SIT_TOT_TURNO'] = eleito

In [ ]:
eleicao3.dropna(subset=['DS_SIT_TOT_TURNO'], inplace=True)

In [ ]:
Counter(eleicao3['DS_SIT_TOT_TURNO'])

**Tratando outros atributos**


In [ ]:
def tratarNaoDivulgavel(coluna):
  print 'Quantos "NÃO DIVULGÁVEL" temos na coluna: ', Counter(eleicao3[coluna])

  moda = eleicao3.loc[:,coluna].mode()[0]

  print '\nPara não modificar muito o valor, colocarei a moda: ', moda, '\n'

  eleicao3[coluna] = [moda if ('NÃO DIVULGÁVEL' in el) else el for el in eleicao3[coluna]]


Genero:

In [ ]:
tratarNaoDivulgavel('DS_GENERO')

eleicao3['DS_GENERO'].values_count()

Cor/raça:

In [ ]:
tratarNaoDivulgavel('DS_COR_RACA')

Counter(eleicao3['DS_COR_RACA'])

Grau de instrução:

In [ ]:
tratarNaoDivulgavel('DS_GRAU_INSTRUCAO')

Counter(eleicao3['DS_GRAU_INSTRUCAO'])

Estado civil:

In [ ]:
tratarNaoDivulgavel('DS_ESTADO_CIVIL')

Counter(eleicao3['DS_ESTADO_CIVIL'])

Nacionalidade:

In [ ]:
tratarNaoDivulgavel('DS_NACIONALIDADE')

Counter(eleicao3['DS_NACIONALIDADE'])

**Análises gráficas em geral**

Com intuito de entender mais a distribuição das caracteristicas dos candidatos

In [ ]:
px.pie(eleicao3, names='TP_AGREMIACAO')

In [ ]:
px.pie(eleicao3, names='DS_NACIONALIDADE')

In [ ]:
px.pie(eleicao3, names='DS_ESTADO_CIVIL')

In [ ]:
px.pie(eleicao3, names='DS_GENERO')

A idade possui valores vazios, vamos analisar esses vazios, e analisar a distribuição de valores em geral:

In [ ]:
print('Quantos nan temos na coluna idade: ', eleicao3['NR_IDADE_DATA_POSSE'].isnull().sum())

media = eleicao3.loc[:,"NR_IDADE_DATA_POSSE"].median()

print('\nPor ser poucos NaNs, iremos pegar a média e colocar no lugar: ', media, '\n')

eleicao3['NR_IDADE_DATA_POSSE'] = [media if pd.isna(el) else el for el in eleicao3['NR_IDADE_DATA_POSSE']]

eleicao3['NR_IDADE_DATA_POSSE'].describe()

In [ ]:
eleicao3.hist(column='NR_IDADE_DATA_POSSE', figsize=(9,6), bins=20);     

In [ ]:
proporcaoGenero = eleicao3.groupby(["DS_SIT_TOT_TURNO", "DS_CARGO", "DS_GENERO"]).size().reset_index(name="QUANTIDADE")

proporcaoGenero.sort_values(by=['DS_SIT_TOT_TURNO'])

In [ ]:
proporcaoGenero = eleicao3.groupby(["DS_SIT_TOT_TURNO", "DS_GENERO"]).size().reset_index(name="QUANTIDADE")

proporcaoGenero.sort_values(by=['DS_SIT_TOT_TURNO'])

In [ ]:
proporcaoEscolaridade = eleicao3.groupby(["DS_SIT_TOT_TURNO","DS_GRAU_INSTRUCAO"]).size().reset_index(name="QUANTIDADE")

proporcaoEscolaridade.sort_values(by=['DS_SIT_TOT_TURNO'])

In [ ]:
nomesSociais = eleicao3['NM_SOCIAL_CANDIDATO'].unique()
nomesSociais = nomesSociais.tolist()

nomesSociais.remove("#NULO#")
nomesSociais.remove("NÃO DIVULGÁVEL")

print("\nTotal dos nomes sociais:", len(nomesSociais))

nomesSociais

In [ ]:
proporcaoRaca = eleicao3.groupby(["DS_SIT_TOT_TURNO","DS_COR_RACA"]).size().reset_index(name="QUANTIDADE")

proporcaoRaca.sort_values(by=['DS_SIT_TOT_TURNO'])

**Preparação para NAIVE BAYES e RANDOM FOREST**

Construo o dataframe final com o label tratado separado dos atributos escolhidos:

In [ ]:
label = eleicao3['DS_SIT_TOT_TURNO']

atributos = pd.DataFrame()
atributos['genero'] = eleicao3['DS_GENERO']  
atributos['estado_civil'] = eleicao3['DS_ESTADO_CIVIL'] 
atributos['cor_raca'] = eleicao3['DS_COR_RACA'] 
atributos['cargo'] = eleicao3['DS_CARGO'] 
atributos['grau_instrucao'] = eleicao3['DS_GRAU_INSTRUCAO'] 

atributos.head()

In [ ]:
print ("{0} Registros e {1} colunas".format(atributos.shape[0], atributos.shape[1]))

Eu transponho os atributos (.T) e transformo em dicionário todos os valores (79873). 

Depois transformo eles em dummies (0 e 1 para cada combinação de valores únicos), pois é a forma que o Naive Bayes aceita de entrada:



In [ ]:
dicionarioAtributos = atributos.T.to_dict().values()
dicionarioAtributos

In [ ]:
vect = DictVectorizer(sparse=False) 

binarizadoAtributos = vect.fit_transform(dicionarioAtributos)

print("Shape após binarizacao/dummerização: {0}".format(binarizadoAtributos.shape), '\n')

binarizadoAtributos

Novas colunas que foram criadas, sendo seus valores agora 0 e 1:

In [ ]:
vect.feature_names_

Binarizo o label também:

In [ ]:
labelEncoder = LabelEncoder()
binarizadoLabel = labelEncoder.fit_transform(label)

print("Labels: {0}, \nClasses: {1}".format(binarizadoLabel, labelEncoder.classes_))

Divido a base em treino e teste, sendo 20% teste e 80% treino. 

O random_state é 0 para sempre começar do mesmo ponto:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(binarizadoAtributos, binarizadoLabel, random_state=0, test_size=0.20)

print ("O shape de cada um é:  {0},  {1},  {2},  {3}".format(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape))

**NAIVE BAYES**

Treino o Naive Bayes com a base de treino:

In [ ]:
naiveBayes = GaussianNB() 
naiveBayes = naiveBayes.fit(X_train, Y_train)

Faço uma previsão com a base de teste:

In [ ]:
Y_predNaiveBayes = naiveBayes.predict(X_test)
Y_predNaiveBayes

Comparo o que foi previsto com os labels reais que tenho e vejo quantos erros ele teve na previsão:

In [ ]:
(Y_test != Y_predNaiveBayes).sum()

Analisando a matriz de confusão:

In [ ]:
matrizConfusao = confusion_matrix(Y_test, Y_predNaiveBayes)
pd.DataFrame(data=matrizConfusao, index=labelEncoder.classes_, columns=[x + "(prev)" for x in labelEncoder.classes_])

Analisando a acurácia do método:

In [ ]:
print("Acurácia no treino:", naiveBayes.score(X_train, Y_train))

print("Acurácia no teste:", naiveBayes.score(X_test, Y_test))

**RANDOM FOREST**

Treino com Random Forest e 10 classificadores:


In [ ]:
randomForest = RandomForestClassifier(n_estimators=10, random_state=0)

randomForest.fit(X_train, Y_train)

Faço a previsão do Y_pred:

In [ ]:
Y_predRandomForest = randomForest.predict(X_test)

Y_predRandomForest

Comparo o que foi previsto com os labels reais que tenho, vendo novamente quantos erros tive:

In [ ]:
(Y_test != Y_predRandomForest).sum()

Análise da acurácia:

In [ ]:
print("Acurácia no treino:", randomForest.score(X_train, Y_train))

print("Acurácia no teste:", randomForest.score(X_test, Y_test))

Matriz de confusão para o Random Forest:

In [ ]:
matrizConfusao = confusion_matrix(Y_test, Y_predRandomForest)
pd.DataFrame(data=matrizConfusao, index=labelEncoder.classes_, columns=[x + "(prev)" for x in labelEncoder.classes_])

**NAIVE BAYES e RANDOM FOREST - 2º TESTE**

Adicionando 2 atributos novos:


In [ ]:
atributos['agremiacao'] = eleicao3['TP_AGREMIACAO']

atributos['nacionalidade'] = eleicao3['DS_NACIONALIDADE']

atributos.head()

Criando funções para não repetir novamente os mesmos códigos:

In [ ]:
def binarizar(atributos):
  dicionarioAtributos = atributos.T.to_dict().values()

  vect = DictVectorizer(sparse=False) 

  binarizadoAtributos = vect.fit_transform(dicionarioAtributos)

  return {
          'vect': vect, 
          'binarizacao' : binarizadoAtributos
          }


def diferenca(Y_test, Y_pred):
  return (Y_test != Y_pred).sum()


def printAcuracia(treino, teste, diferenca):
  print("Acurácia no treino {0}, no teste {1}, diferença {2}\n".format (treino, teste, diferenca))

Nova binarização com mais variaveis:

In [ ]:
resultado = binarizar(atributos)

resultado['vect'].feature_names_

In [ ]:
def treinarPreverMedir(binarizacao, binarizadoLabel, estimatores = 10, test_size=0.30):
  
  X_train, X_test, Y_train, Y_test = train_test_split(binarizacao, binarizadoLabel, random_state=0, test_size=test_size)

  #Naive Bayes
  naiveBayes = GaussianNB() 
  naiveBayes = naiveBayes.fit(X_train, Y_train)

  Y_predNaiveBayes = naiveBayes.predict(X_test)

  #Random Forest
  randomForest = RandomForestClassifier(estimatores, random_state=0)

  randomForest.fit(X_train, Y_train)

  Y_predRandomForest = randomForest.predict(X_test)

  #Análises
  printAcuracia(naiveBayes.score(X_train, Y_train), naiveBayes.score(X_test, Y_test), diferenca(Y_test, Y_predNaiveBayes))

  printAcuracia(randomForest.score(X_train, Y_train), randomForest.score(X_test, Y_test), diferenca(Y_test, Y_predRandomForest))

In [ ]:
treinarPreverMedir(resultado['binarizacao'], binarizadoLabel)

Misturando aleatoriamente os registros do dataframe:

In [ ]:
atributos1 = atributos.sample(frac=1)
atributos1.head()

Prevendo com a mistura da base:

In [ ]:
resultado = binarizar(atributos1)

treinarPreverMedir(resultado['binarizacao'], binarizadoLabel)